In [ ]:
%load_ext autoreload
%autoreload 2

from skorecard import datasets

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score


# from skorecard.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegression

# from feature_engine.discretisers import EqualWidthDiscretiser

import pandas as pd
import numpy as np

### Load Data

In [ ]:
X, y = datasets.load_uci_credit_card(return_X_y=True)

In [ ]:
X.head()

In [ ]:
X['EDUCATION'].value_counts()

In [ ]:
X['MARRIAGE'].value_counts()

In [ ]:
X['LIMIT_BAL'].describe()

In [ ]:
X['BILL_AMT1'].describe()

### Specify categorical and numerical columns

#### We'll autodetect using dabl.detect_types

In [ ]:
from dabl import detect_types

In [ ]:
detected_types = detect_types(X)
detected_types

In [ ]:
cat_columns = X.columns[(detected_types['categorical']==True) | (detected_types['low_card_int']==True)]

In [ ]:
num_columns = X.columns[(detected_types['continuous']==True) | (detected_types['dirty_float']==True)]
print(f"cat_columns = {cat_columns}")
print(f"num_columns = {num_columns}")

## Bucketers

### EqualWidthBucketer

In [ ]:
from skorecard.bucketers import EqualWidthBucketer

In [ ]:
EWB = EqualWidthBucketer(bins = 5) # show non-int

In [ ]:
EWB.fit(X['LIMIT_BAL']) # this breaks - needs pandas df

In [ ]:
EWB.fit(X[['LIMIT_BAL']])

In [ ]:
EWB.bins

In [ ]:
EWB.bucketer # Note the equal (well, almost) widths!

In [ ]:
EWB.features_bucket_mapping_

In [ ]:
X_transform = EWB.transform(X[['LIMIT_BAL']])

In [ ]:
X_transform

In [ ]:
X_transform['LIMIT_BAL'].value_counts()

### EqualFrequencyBucketer

In [ ]:
from skorecard.bucketers import EqualFrequencyBucketer

In [ ]:
EFB = EqualFrequencyBucketer(bins = 5)

In [ ]:
EFB.fit(X[['BILL_AMT1']])

In [ ]:
EFB.variables

In [ ]:
EFB.bucketer # Note the counts

In [ ]:
X_transform = EFB.transform(X[['BILL_AMT1']])

In [ ]:
X_transform.head()

In [ ]:
X_transform.value_counts()

#### Not always perfect: LIMIT_BAL

In [ ]:
EFB = EqualFrequencyBucketer(bins = 5)
EFB.fit(X[['LIMIT_BAL']])
EFB.bucketer # Note the counts

### Agglomerative

In [ ]:
from skorecard.bucketers import AgglomerativeClusteringBucketer

In [ ]:
ACB = AgglomerativeClusteringBucketer(bins = 5)

In [ ]:
ACB.fit(X[['BILL_AMT1']])

In [ ]:
ACB.bucketer

In [ ]:
X_transform = ACB.transform(X[['BILL_AMT1']])

In [ ]:
X_transform.head()

In [ ]:
X_transform.value_counts()

### Categoricals

In [ ]:
from skorecard.bucketers import OrdinalCategoricalBucketer

In [ ]:
OCB = OrdinalCategoricalBucketer(bins=3) # this fails

In [ ]:
X['MARRIAGE'].value_counts(normalize=True)

In [ ]:
OCB = OrdinalCategoricalBucketer(tol=0.15, max_n_categories=None)

In [ ]:
OCB.fit_transform(X[['BILL_AMT1']])

In [ ]:
OCB = OrdinalCategoricalBucketer(tol=0.15, max_n_categories=None)

In [ ]:
OCB.fit(X[['MARRIAGE']])

In [ ]:
X_transform = OCB.transform(X[['MARRIAGE']])

In [ ]:
X_transform.value_counts(normalize=True)

### And if we increase tol?

In [ ]:
OCB = OrdinalCategoricalBucketer(tol=0.50, max_n_categories=None)
OCB.fit(X[['MARRIAGE']])
X_transform = OCB.transform(X[['MARRIAGE']])
X_transform.value_counts(normalize=True)

### DecisionTreeBucketer

In [ ]:
from skorecard.bucketers import DecisionTreeBucketer

In [ ]:
DTB = DecisionTreeBucketer()

In [ ]:
DTB.fit(X[['LIMIT_BAL']], y)

In [ ]:
X_transform = DTB.transform(X[['LIMIT_BAL']])

In [ ]:
X_transform['LIMIT_BAL'].value_counts()

In [ ]:
DTB.bucketer

In [ ]:
DTB.features_bucket_mapping_

In [ ]:
DTB.get_params()

## Making a pipeline

In [ ]:
bucket_pipeline = make_pipeline(
    EqualWidthBucketer(bins=5, variables=list(num_columns)),
    OrdinalCategoricalBucketer(variables=list(cat_columns))
)

In [ ]:
bucket_pipeline

In [ ]:
bucket_pipeline.named_steps

In [ ]:
pipeline = Pipeline([
    ('bucketing', bucket_pipeline),
    ('one-hot-encoding', OneHotEncoder()),
    ('lr', LogisticRegression())
])

In [ ]:
pipeline

In [ ]:
pipeline.named_steps

In [ ]:
pipeline.fit(X, y)
f"AUC = {roc_auc_score(y, pipeline.predict_proba(X)[:,1]):.4f}"

In [ ]:
bucket_pipeline = make_pipeline(
    DecisionTreeBucketer(max_depth=4,min_samples_leaf=0.1, variables=list(num_columns)),
    OrdinalCategoricalBucketer(variables=list(cat_columns))
)

pipeline = Pipeline([
    ('bucketing', bucket_pipeline),
    ('one-hot-encoding', OneHotEncoder()),
    ('lr', LogisticRegression())
])

pipeline.fit(X, y)
f"AUC = {roc_auc_score(y, pipeline.predict_proba(X)[:,1]):.4f}"

In [ ]:
cat_columns

In [ ]:
# Using a ColumnTransformer


bucket_pipeline = ColumnTransformer([
    ('categorical_preprocessing', OrdinalCategoricalBucketer(),cat_columns),
], remainder="passthrough")

ohe_pipeline = ColumnTransformer([
    ('ohe_cat_preprocessing', OneHotEncoder(),[0,1]),
], remainder="passthrough")

# bucket_pipeline.fit_transform(X, y)

pipeline = Pipeline([
    ('bucketing', bucket_pipeline),
    ('ohe', ohe_pipeline),
    ('lr', LogisticRegression())
])

pipeline.fit(X, y)
f"AUC = {roc_auc_score(y, pipeline.predict_proba(X)[:,1]):.4f}"

In [ ]:
# Using a ColumnTransformer


bucket_pipeline = ColumnTransformer([
    ('categorical_preprocessing', OrdinalCategoricalBucketer(),cat_columns),
    ('numerical_preprocessing', DecisionTreeBucketer(max_depth=4,min_samples_leaf=0.1), ['LIMIT_BAL','BILL_AMT1'])
], remainder="passthrough")

ohe_pipeline = ColumnTransformer([
    ('ohe_cat_preprocessing', OneHotEncoder(),[0,1]),
], remainder="passthrough")

# bucket_pipeline.fit_transform(X, y)

pipeline = Pipeline([
    ('bucketing', bucket_pipeline),
    ('ohe', ohe_pipeline),
    ('lr', LogisticRegression())
])

pipeline.fit(X, y)
f"AUC = {roc_auc_score(y, pipeline.predict_proba(X)[:,1]):.4f}"